In [306]:
import networkx as nx
import pandas as pd
import pickle
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt 
import seaborn as sns
import scipy.spatial.distance as spsd
import scipy.linalg as spla
import sys
import random
import math
import itertools
import operator
from sklearn.metrics import mean_squared_error

In [307]:
def style_specific_cell(x):
    global color_pos 
    color = 'background-color: red'
    df1 = pd.DataFrame('', index=x.index, columns=x.columns)
    for x in color_pos:
        df1.iloc[x[0],x[1]] = color
    return df1

In [308]:
#path='/Users/app/Desktop/Net/NetSampling/network_sampling'
path="/Users/app/Desktop/Net/network_sampling"
result=list(Path(path).glob("*"+'.pkl'))
len(result)


594

In [309]:
Name=['RN','DN','PN','RE','RNE','HYB','SBS','FFS_Geom','BFS','DFS','RW','MHRW','FS','SP']
Name_List=[]
for name in Name:
    Name_List.append(name)
    if name in ['RE','RNE','HYB','SBS','FFS_Geom','BFS','DFS','RW','MHRW','FS']:
        Name_List.append('Induced_'+name)
    

In [310]:
result[0:5]

[PosixPath('/Users/app/Desktop/Net/network_sampling/BA_high_community_5.pkl'),
 PosixPath('/Users/app/Desktop/Net/network_sampling/SBM_high_degree_1.pkl'),
 PosixPath('/Users/app/Desktop/Net/network_sampling/Graphon_mid_community_1.pkl'),
 PosixPath('/Users/app/Desktop/Net/network_sampling/Graphon_mid_eigenvalue_1.pkl'),
 PosixPath('/Users/app/Desktop/Net/network_sampling/WS_high_degree_5.pkl')]

In [311]:
node={}
for a in ['ER','SBM','WS','BA','Graphon']:
    node[str(a)]={}
    for b in ['low','mid','high']:
        node[str(a)][str(b)]={}
        #for c in ['degree','clustering','community','eigenvalue']:
            #node[str(a)][str(b)][str(c)]={}
node={}
for a in ['ER','SBM','WS','BA','Graphon']:
    node[str(a)]={}
    for b in ['low','mid','high']:
        node[str(a)][str(b)]={}
        for c in ['degree','clustering','community','eigenvalue']:
            node[str(a)][str(b)][str(c)]=[str(x )for x in result if f"{a}_{b}_{c}" in str(x)]
            if len(node[str(a)][str(b)][str(c)])!=10:
                print(f'{a,b,c} incomplete')
            


('ER', 'high', 'clustering') incomplete


In [312]:
#for path in node['BA']['low']['clustering']:
with open(node['Graphon']['low']['community'][0],'rb') as f:
    r=pickle.load(f)
len(r[0])

24

In [313]:
#community number
df_NMSE=pd.DataFrame({})
df_Bias=pd.DataFrame({})
df_Var=pd.DataFrame({})
DF=pd.DataFrame({})
for index,model in enumerate(['ER','SBM','WS','BA','Graphon']):
    df_nmse=pd.DataFrame({})
    df_bias=pd.DataFrame({})
    df_var=pd.DataFrame({})
    for method in range(24):
        NMSE=[];Var=[];Bias=[]
        for density in ['low','mid','high']:
            AveD=[];Density_SG=[];Degree_Dist=[]
            for path in node[model][density]['community']:
                with open(path,'rb') as f:
                    r=pickle.load(f)
                re=r[0][method]
                AveD=AveD+re[0][0]
                #Degree_Dist=Degree_Dist+re[1]
                #Density_SG=Density_SG+re[2]
            real=re[0][1]
            y_true = [real for i in  AveD]
            y_pred =  AveD
            bias = np.mean(y_pred)-np.mean(y_true)
            var = np.var(y_pred)
            nmse = math.sqrt(mean_squared_error(y_true, y_pred))/real
            NMSE.append(nmse);Bias.append(bias);Var.append(var)
        #Degreedist_List.append(Degree_Dist)
        #Density_List.append(Density_SG)

        df_nmse=pd.concat([df_nmse,pd.DataFrame([[np.mean(NMSE)]],columns=['nmse'])])
        df_bias=pd.concat([df_bias,pd.DataFrame([[np.mean(Bias)]],columns=['bias'])])
        df_var=pd.concat([df_var,pd.DataFrame([[np.mean(Var)]],columns=['var'])])
            #df_nmse=pd.DataFrame([[np.mean(NMSE),np.mean(Bias),np.mean(Var)]],columns=['nmse',"bias",'variance'])
            
    
    df_NMSE=pd.concat([df_NMSE,df_nmse], axis=1) 
    df_Var=pd.concat([df_Var,df_var], axis=1) 
    df_Bias=pd.concat([df_Bias,df_bias], axis=1) 
#df_NMSE   
#df_total.columns=['ER_NMSE','SBM_NMSE','WS_NMSE','BA_NMSE','Graphon_NMSE','ER_Bias','SBM_Bias','WS_Bias','BA_Bias','Graphon_Bias','ER_Variance','SBM_Variance','WS_Variance','BA_Variance','Graphon_Variance']
#df_total.insert(0, "Method", Name_List)
#df_total.index=range(24)
DF=pd.concat([df_NMSE,df_Bias,df_Var],axis=1)

#df_NMSE.columns=["ER_NMSE","SBM_NMSE",'WS_NMSE','BA_NMSE','Graphon_NMSE']
#df_NMSE.index=range(24)
#df_NMSE.insert(0,"Method",Name_List)

In [314]:
DF.columns=['ER_NMSE','SBM_NMSE','WS_NMSE','BA_NMSE','Graphon_NMSE','ER_Bias','SBM_Bias','WS_Bias','BA_Bias','Graphon_Bias','ER_Variance','SBM_Variance','WS_Variance','BA_Variance','Graphon_Variance']
DF.index=range(24)
DF.insert(0,"Method",Name_List)       
  

In [315]:
color_pos=[]
for x in range(1,len(DF.columns)):
    color_pos.append([DF.iloc[list(DF.index),x].abs().sort_values().index[0:3],x])
DF_=DF.style.apply(style_specific_cell, axis=None)
DF_.set_precision(5)
display(DF_)

,Method,ER_NMSE,SBM_NMSE,WS_NMSE,BA_NMSE,Graphon_NMSE,ER_Bias,SBM_Bias,WS_Bias,BA_Bias,Graphon_Bias,ER_Variance,SBM_Variance,WS_Variance,BA_Variance,Graphon_Variance
0,RN,8.86228,0.69107,8.09325,25.52016,1.57658,164.28133,2.01333,557.80500,487.99500,6.75233,148.81036,0.73413,255.12859,627.83229,2.16871
1,DN,7.33444,0.59466,7.77620,6.56119,1.55327,135.23867,1.72767,536.03300,160.08400,6.62700,139.66121,0.59317,258.07020,202.98601,2.07808
2,PN,7.58687,0.62684,7.82832,7.71126,1.55468,140.05500,1.83033,539.63633,183.57200,6.64000,145.59049,0.55864,244.09252,232.58545,2.13111
3,RE,46.75757,156.83138,5.95444,27.94607,104.18650,461.74733,470.47900,408.04033,419.47533,468.65233,14.82520,14.25931,12.76690,39.63430,13.65470
4,Induced_RE,1.71543,0.37858,2.62780,0.95620,1.44765,28.62300,1.08300,181.57533,25.99267,6.09167,27.34116,0.35100,128.93386,33.09057,1.71017
5,RNE,46.81523,156.85390,5.96654,29.45657,104.22903,462.63567,470.54633,408.87267,443.24500,468.85700,13.88471,14.45213,12.34188,20.28544,14.76702
6,Induced_RNE,2.01142,0.45301,2.72517,1.87521,1.46800,34.24133,1.31300,188.30700,49.11867,6.18000,33.10550,0.36909,125.73103,66.19606,1.82673
7,HYB,46.80541,156.85234,5.96479,29.20638,104.25380,462.50600,470.54133,408.75467,439.25800,468.96000,14.47046,14.76804,12.30485,23.23373,14.41839
8,Induced_HYB,1.95595,0.43725,2.70381,1.64170,1.47127,33.15600,1.26333,186.84167,43.40633,6.20300,32.41091,0.37397,123.22749,55.33149,1.85698
9,SBS,1.98960,8.89996,0.54804,1.08410,5.60133,17.80167,26.58267,-37.61167,9.61667,24.92833,5.91332,6.24577,4.26861,4.78485,6.41558


In [333]:
#largest eigen value
df_NMSE=pd.DataFrame({})
df_Bias=pd.DataFrame({})
df_Var=pd.DataFrame({})
DF=pd.DataFrame({})
for index,model in enumerate(['ER','SBM','WS','BA','Graphon']):
    df_nmse=pd.DataFrame({})
    df_bias=pd.DataFrame({})
    df_var=pd.DataFrame({})
    for method in range(24):
        NMSE=[];Var=[];Bias=[]
        for density in ['low','mid','high']:
            AveD=[];Density_SG=[];Degree_Dist=[]
            for path in node[model][density]['eigenvalue']:
                with open(path,'rb') as f:
                    r=pickle.load(f)
                re=r[0][method]
                AveD=AveD+re[0]
                #Degree_Dist=Degree_Dist+re[1]
                #Density_SG=Density_SG+re[2]
            real=re[3][0]/10000
            y_true = [real for i in  AveD]
            y_pred =  AveD
            bias = np.mean(y_pred)-np.mean(y_true)
            var = np.var(y_pred)
            nmse = math.sqrt(mean_squared_error(y_true, y_pred))/real
            NMSE.append(nmse);Bias.append(bias);Var.append(var)
        #Degreedist_List.append(Degree_Dist)
        #Density_List.append(Density_SG)

        df_nmse=pd.concat([df_nmse,pd.DataFrame([[np.mean(NMSE)]],columns=['nmse'])])
        df_bias=pd.concat([df_bias,pd.DataFrame([[np.mean(Bias)]],columns=['bias'])])
        df_var=pd.concat([df_var,pd.DataFrame([[np.mean(Var)*10**6]],columns=['var'])])
            #df_nmse=pd.DataFrame([[np.mean(NMSE),np.mean(Bias),np.mean(Var)]],columns=['nmse',"bias",'variance'])
            
    
    df_NMSE=pd.concat([df_NMSE,df_nmse], axis=1) 
    df_Var=pd.concat([df_Var,df_var], axis=1) 
    df_Bias=pd.concat([df_Bias,df_bias], axis=1) 

DF=pd.concat([df_NMSE,df_Bias,df_Var],axis=1)


In [332]:
DF.columns=['ER_NMSE','SBM_NMSE','WS_NMSE','BA_NMSE','Graphon_NMSE','ER_Bias','SBM_Bias','WS_Bias','BA_Bias','Graphon_Bias','ER_Variance','SBM_Variance','WS_Variance','BA_Variance','Graphon_Variance']
DF.index=range(24)
DF.insert(0,"Method",Name_List) 
color_pos=[]
for x in range(1,len(DF.columns)):
    color_pos.append([DF.iloc[list(DF.index),x].abs().sort_values().index[0:3],x])
DF_=DF.style.apply(style_specific_cell, axis=None)
DF_.set_precision(5)
display(DF_)

,Method,ER_NMSE,SBM_NMSE,WS_NMSE,BA_NMSE,Graphon_NMSE,ER_Bias,SBM_Bias,WS_Bias,BA_Bias,Graphon_Bias,ER_Variance,SBM_Variance,WS_Variance,BA_Variance,Graphon_Variance
0,RN,0.52110,0.07342,2.92040,0.78962,0.07046,0.00111,0.00111,0.00211,0.00214,0.00093,0.07355,0.58967,0.09514,1.06903,0.04339
1,DN,0.57314,0.11487,2.99648,3.73048,0.08450,0.00127,0.00235,0.00216,0.01227,0.00113,0.06963,1.67806,0.09364,0.55221,0.04333
2,PN,0.56354,0.08968,2.98030,3.50399,0.08215,0.00124,0.00151,0.00215,0.01145,0.00110,0.07086,0.84855,0.09835,0.69964,0.04158
3,RE,0.80517,0.88842,1.52341,0.47564,0.87185,-0.03533,-0.03489,0.00092,-0.00063,-0.01940,0.01178,0.01146,0.01358,0.09794,0.01096
4,Induced_RE,0.69386,0.15957,3.90876,3.76110,0.13811,0.00188,0.00310,0.00287,0.01237,0.00190,0.07440,2.54241,0.11641,0.58426,0.04654
5,RNE,0.79900,0.88901,1.51218,0.42976,0.87232,-0.03534,-0.03490,0.00092,-0.00116,-0.01940,0.01115,0.01123,0.01477,0.06283,0.01057
6,Induced_RNE,0.66444,0.13007,3.87582,2.93518,0.13063,0.00179,0.00219,0.00284,0.00933,0.00179,0.07338,1.22010,0.11797,1.02684,0.04345
7,HYB,0.79942,0.88900,1.50894,0.44085,0.87201,-0.03534,-0.03490,0.00092,-0.00105,-0.01940,0.01073,0.01104,0.01440,0.07070,0.01074
8,Induced_HYB,0.66958,0.13572,3.87653,3.14715,0.13184,0.00182,0.00237,0.00285,0.01011,0.00181,0.07133,1.53110,0.11624,0.85491,0.04533
9,SBS,1.17500,0.80054,3.48417,0.31520,0.77004,-0.03392,-0.03349,0.00231,-0.00016,-0.01798,0.00000,0.00000,0.00028,0.00027,0.00000


In [336]:
#2nd Largest eigenvalue
df_NMSE=pd.DataFrame({})
df_Bias=pd.DataFrame({})
df_Var=pd.DataFrame({})
DF=pd.DataFrame({})
for index,model in enumerate(['ER','SBM','WS','BA','Graphon']):
    df_nmse=pd.DataFrame({})
    df_bias=pd.DataFrame({})
    df_var=pd.DataFrame({})
    for method in range(24):
        NMSE=[];Var=[];Bias=[]
        for density in ['low','mid','high']:
            AveD=[];Density_SG=[];Degree_Dist=[]
            for path in node[model][density]['eigenvalue']:
                with open(path,'rb') as f:
                    r=pickle.load(f)
                re=r[0][method]
                AveD=AveD+re[1]
                #Degree_Dist=Degree_Dist+re[1]
                #Density_SG=Density_SG+re[2]
            real=re[3][1]/10000
            y_true = [real for i in  AveD]
            y_pred =  AveD
            bias = np.mean(y_pred)-np.mean(y_true)
            var = np.var(y_pred)
            nmse = math.sqrt(mean_squared_error(y_true, y_pred))/real
            NMSE.append(nmse);Bias.append(bias);Var.append(var)
        #Degreedist_List.append(Degree_Dist)
        #Density_List.append(Density_SG)

        df_nmse=pd.concat([df_nmse,pd.DataFrame([[np.mean(NMSE)]],columns=['nmse'])])
        df_bias=pd.concat([df_bias,pd.DataFrame([[np.mean(Bias)]],columns=['bias'])])
        df_var=pd.concat([df_var,pd.DataFrame([[np.mean(Var)*10**7]],columns=['var'])])
            #df_nmse=pd.DataFrame([[np.mean(NMSE),np.mean(Bias),np.mean(Var)]],columns=['nmse',"bias",'variance'])
            
    
    df_NMSE=pd.concat([df_NMSE,df_nmse], axis=1) 
    df_Var=pd.concat([df_Var,df_var], axis=1) 
    df_Bias=pd.concat([df_Bias,df_bias], axis=1) 

DF=pd.concat([df_NMSE,df_Bias,df_Var],axis=1)
DF.columns=['ER_NMSE','SBM_NMSE','WS_NMSE','BA_NMSE','Graphon_NMSE','ER_Bias','SBM_Bias','WS_Bias','BA_Bias','Graphon_Bias','ER_Variance','SBM_Variance','WS_Variance','BA_Variance','Graphon_Variance']
DF.index=range(24)
DF.insert(0,"Method",Name_List) 
color_pos=[]
for x in range(1,len(DF.columns)):
    color_pos.append([DF.iloc[list(DF.index),x].abs().sort_values().index[0:3],x])
DF_=DF.style.apply(style_specific_cell, axis=None)
DF_.set_precision(5)
display(DF_)

,Method,ER_NMSE,SBM_NMSE,WS_NMSE,BA_NMSE,Graphon_NMSE,ER_Bias,SBM_Bias,WS_Bias,BA_Bias,Graphon_Bias,ER_Variance,SBM_Variance,WS_Variance,BA_Variance,Graphon_Variance
0,RN,2.44926,0.09633,3.01401,1.31897,0.59736,0.00642,0.00083,0.00198,0.00243,0.00281,0.10748,23.16099,0.41256,3.21099,0.55586
1,DN,2.53398,0.11426,3.09848,3.27587,0.60901,0.00649,0.00137,0.00204,0.00603,0.00287,0.09368,21.45422,0.40086,0.54846,0.58290
2,PN,2.51886,0.10667,3.07806,3.15029,0.60726,0.00648,0.00111,0.00202,0.00580,0.00286,0.09480,22.40085,0.41996,0.64726,0.56696
3,RE,0.81133,0.87575,1.64049,0.41743,0.65275,-0.00118,-0.02956,0.00095,0.00047,-0.00558,0.06285,0.06490,0.08342,0.35762,0.06367
4,Induced_RE,2.72945,0.15959,4.11601,3.30805,0.65474,0.00669,0.00205,0.00272,0.00609,0.00309,0.09221,32.19005,0.57697,0.55235,0.60332
5,RNE,0.80047,0.87633,1.61898,0.29035,0.65375,-0.00119,-0.02956,0.00094,0.00005,-0.00558,0.06862,0.06553,0.08549,0.19737,0.06383
6,Induced_RNE,2.68261,0.14383,4.07866,2.88642,0.64815,0.00665,0.00155,0.00270,0.00529,0.00306,0.09207,29.77448,0.56323,0.84232,0.58860
7,HYB,0.80237,0.87621,1.62211,0.31076,0.65319,-0.00119,-0.02956,0.00094,0.00013,-0.00558,0.06625,0.06356,0.08705,0.25572,0.06524
8,Induced_HYB,2.69186,0.14752,4.07770,2.98568,0.64922,0.00666,0.00167,0.00270,0.00548,0.00307,0.08677,30.69913,0.56909,0.76176,0.57271
9,SBS,1.57916,0.77238,3.85109,0.73450,0.38851,0.00024,-0.02814,0.00236,0.00120,-0.00415,0.00001,0.00000,0.00196,0.00191,0.00000


In [337]:
#3rd Largest eigenvalue
df_NMSE=pd.DataFrame({})
df_Bias=pd.DataFrame({})
df_Var=pd.DataFrame({})
DF=pd.DataFrame({})
for index,model in enumerate(['ER','SBM','WS','BA','Graphon']):
    df_nmse=pd.DataFrame({})
    df_bias=pd.DataFrame({})
    df_var=pd.DataFrame({})
    for method in range(24):
        NMSE=[];Var=[];Bias=[]
        for density in ['low','mid','high']:
            AveD=[];Density_SG=[];Degree_Dist=[]
            for path in node[model][density]['eigenvalue']:
                with open(path,'rb') as f:
                    r=pickle.load(f)
                re=r[0][method]
                AveD=AveD+re[2]
                #Degree_Dist=Degree_Dist+re[1]
                #Density_SG=Density_SG+re[2]
            real=re[3][2]/10000
            y_true = [real for i in  AveD]
            y_pred =  AveD
            bias = np.mean(y_pred)-np.mean(y_true)
            var = np.var(y_pred)
            nmse = math.sqrt(mean_squared_error(y_true, y_pred))/real
            NMSE.append(nmse);Bias.append(bias);Var.append(var)
        #Degreedist_List.append(Degree_Dist)
        #Density_List.append(Density_SG)

        df_nmse=pd.concat([df_nmse,pd.DataFrame([[np.mean(NMSE)]],columns=['nmse'])])
        df_bias=pd.concat([df_bias,pd.DataFrame([[np.mean(Bias)]],columns=['bias'])])
        df_var=pd.concat([df_var,pd.DataFrame([[np.mean(Var)*10**7]],columns=['var'])])
            #df_nmse=pd.DataFrame([[np.mean(NMSE),np.mean(Bias),np.mean(Var)]],columns=['nmse',"bias",'variance'])
            
    
    df_NMSE=pd.concat([df_NMSE,df_nmse], axis=1) 
    df_Var=pd.concat([df_Var,df_var], axis=1) 
    df_Bias=pd.concat([df_Bias,df_bias], axis=1) 

DF=pd.concat([df_NMSE,df_Bias,df_Var],axis=1)
DF.columns=['ER_NMSE','SBM_NMSE','WS_NMSE','BA_NMSE','Graphon_NMSE','ER_Bias','SBM_Bias','WS_Bias','BA_Bias','Graphon_Bias','ER_Variance','SBM_Variance','WS_Variance','BA_Variance','Graphon_Variance']
DF.index=range(24)
DF.insert(0,"Method",Name_List) 
color_pos=[]
for x in range(1,len(DF.columns)):
    color_pos.append([DF.iloc[list(DF.index),x].abs().sort_values().index[0:3],x])
DF_=DF.style.apply(style_specific_cell, axis=None)
DF_.set_precision(5)
display(DF_)

,Method,ER_NMSE,SBM_NMSE,WS_NMSE,BA_NMSE,Graphon_NMSE,ER_Bias,SBM_Bias,WS_Bias,BA_Bias,Graphon_Bias,ER_Variance,SBM_Variance,WS_Variance,BA_Variance,Graphon_Variance
0,RN,2.39983,0.09347,2.86777,1.17123,2.20808,0.00631,0.00065,0.00186,0.00208,0.00593,0.07381,6.68516,0.28406,1.92008,0.05804
1,DN,2.48693,0.10556,2.95670,3.18517,2.22934,0.00639,0.00107,0.00191,0.00562,0.00598,0.06647,2.88102,0.26602,0.41059,0.05915
2,PN,2.46934,0.10265,2.94129,3.05843,2.22632,0.00637,0.00092,0.00190,0.00539,0.00597,0.06631,4.38087,0.27621,0.46522,0.05585
3,RE,0.80043,0.86697,1.58708,0.39504,0.32698,-0.00122,-0.02666,0.00091,0.00039,-0.00105,0.07031,0.06773,0.09346,0.21692,0.06723
4,Induced_RE,2.68265,0.14447,3.96789,3.21935,2.30455,0.00658,0.00137,0.00258,0.00567,0.00615,0.06331,6.64834,0.40106,0.38793,0.05828
5,RNE,0.78811,0.86771,1.56245,0.28023,0.32817,-0.00123,-0.02666,0.00089,0.00003,-0.00105,0.07525,0.06601,0.09672,0.11821,0.06672
6,Induced_RNE,2.63754,0.13660,3.92541,2.78830,2.29368,0.00654,0.00108,0.00255,0.00489,0.00612,0.06198,11.75478,0.39302,0.70475,0.05771
7,HYB,0.78891,0.86758,1.56407,0.29952,0.32770,-0.00123,-0.02666,0.00089,0.00010,-0.00105,0.07449,0.06691,0.09644,0.16001,0.06929
8,Induced_HYB,2.64749,0.13865,3.93251,2.89331,2.29530,0.00655,0.00116,0.00256,0.00509,0.00613,0.06352,10.33614,0.39490,0.59314,0.05921
9,SBS,1.57652,0.75057,3.87476,0.80427,0.41774,0.00023,-0.02521,0.00235,0.00125,0.00040,0.00003,0.00000,0.00245,0.00160,0.00000


In [294]:
#density
df_NMSE=pd.DataFrame({})
df_Bias=pd.DataFrame({})
df_Var=pd.DataFrame({})
DF=pd.DataFrame({})
for index,model in enumerate(['ER','SBM','WS','BA','Graphon']):
    df_nmse=pd.DataFrame({})
    df_bias=pd.DataFrame({})
    df_var=pd.DataFrame({})
    for method in range(24):
        NMSE=[];Var=[];Bias=[]
        for density in ['low','mid','high']:
            AveD=[];Density_SG=[];Degree_Dist=[]
            for path in node[model][density]['degree']:
                with open(path,'rb') as f:
                    r=pickle.load(f)
                re=r[0][method]
                AveD=AveD+re[2][0]
                #Degree_Dist=Degree_Dist+re[1]
                #Density_SG=Density_SG+re[2]
            real=re[2][1]
            y_true = [real for i in  AveD]
            y_pred =  AveD
            bias = np.mean(y_pred)-np.mean(y_true)
            var = np.var(y_pred)
            nmse = math.sqrt(mean_squared_error(y_true, y_pred))/real
            NMSE.append(nmse);Bias.append(bias);Var.append(var)
        #Degreedist_List.append(Degree_Dist)
        #Density_List.append(Density_SG)

        df_nmse=pd.concat([df_nmse,pd.DataFrame([[np.mean(NMSE)]],columns=['nmse'])])
        df_bias=pd.concat([df_bias,pd.DataFrame([[np.mean(Bias)]],columns=['bias'])])
        df_var=pd.concat([df_var,pd.DataFrame([[np.mean(Var)]],columns=['var'])])
            #df_nmse=pd.DataFrame([[np.mean(NMSE),np.mean(Bias),np.mean(Var)]],columns=['nmse',"bias",'variance'])
            
    
    df_NMSE=pd.concat([df_NMSE,df_nmse], axis=1) 
    df_Var=pd.concat([df_Var,df_var], axis=1) 
    df_Bias=pd.concat([df_Bias,df_bias], axis=1) 

DF=pd.concat([df_NMSE,df_Bias,df_Var],axis=1)
DF.columns=['ER_NMSE','SBM_NMSE','WS_NMSE','BA_NMSE','Graphon_NMSE','ER_Bias','SBM_Bias','WS_Bias','BA_Bias','Graphon_Bias','ER_Variance','SBM_Variance','WS_Variance','BA_Variance','Graphon_Variance']
DF.index=range(24)
DF.insert(0,"Method",Name_List) 
color_pos=[]
for x in range(1,len(DF.columns)):
    color_pos.append([DF.iloc[list(DF.index),x].abs().sort_values().index[0:3],x])
DF_=DF.style.apply(style_specific_cell, axis=None)
DF_.set_precision(5)
display(DF_)

,Method,ER_NMSE,SBM_NMSE,WS_NMSE,BA_NMSE,Graphon_NMSE,ER_Bias,SBM_Bias,WS_Bias,BA_Bias,Graphon_Bias,ER_Variance,SBM_Variance,WS_Variance,BA_Variance,Graphon_Variance
0,RN,0.02151,0.01041,0.04929,0.09416,0.01132,-0.00000,0.00000,-0.00000,-0.00000,-0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
1,DN,0.07743,0.02047,0.09034,2.66739,0.01866,0.00019,0.00036,0.00005,0.00304,0.00020,0.00000,0.00000,0.00000,0.00000,0.00000
2,PN,0.06436,0.01561,0.07906,2.32311,0.01639,0.00016,0.00020,0.00004,0.00263,0.00017,0.00000,0.00000,0.00000,0.00000,0.00000
3,RE,0.64578,0.93281,0.68165,0.77597,0.92339,-0.03596,-0.03520,0.00025,-0.00008,-0.02002,0.00000,0.00000,0.00000,0.00000,0.00000
4,Induced_RE,0.38598,0.07192,1.38303,3.57847,0.07662,0.00100,0.00119,0.00090,0.00376,0.00103,0.00000,0.00000,0.00000,0.00000,0.00000
5,RNE,0.64390,0.93285,0.67895,0.75343,0.92342,-0.03597,-0.03520,0.00025,-0.00013,-0.02002,0.00000,0.00000,0.00000,0.00000,0.00000
6,Induced_RNE,0.35048,0.06075,1.34920,2.61802,0.06969,0.00091,0.00092,0.00088,0.00254,0.00094,0.00000,0.00000,0.00000,0.00000,0.00000
7,HYB,0.64427,0.93285,0.67946,0.75771,0.92339,-0.03596,-0.03520,0.00025,-0.00012,-0.02002,0.00000,0.00000,0.00000,0.00000,0.00000
8,Induced_HYB,0.35602,0.06321,1.35673,2.82321,0.07073,0.00093,0.00097,0.00089,0.00280,0.00095,0.00000,0.00000,0.00000,0.00000,0.00000
9,SBS,0.92526,0.87253,2.05525,1.55630,0.85463,-0.03502,-0.03425,0.00120,0.00080,-0.01907,0.00000,0.00000,0.00000,0.00000,0.00000


In [239]:
df_NMSE=pd.DataFrame({})
df_Bias=pd.DataFrame({})
df_Var=pd.DataFrame({})
for index,model in enumerate(['ER','SBM','WS','BA','Graphon']):
    df_nmse=pd.DataFrame({})
    df_bias=pd.DataFrame({})
    df_var=pd.DataFrame({})
    for method in range(24):
        NMSE=[];Var=[];Bias=[]
        for density in ['low','mid','high']:
            AveD=[];Density_SG=[];Degree_Dist=[]
            for path in node[model][density]['degree']:
                with open(path,'rb') as f:
                    r=pickle.load(f)
                re=r[0][method]
                AveD=AveD+re[2][0]
                #Degree_Dist=Degree_Dist+re[1]
                #Density_SG=Density_SG+re[2]
            real=re[2][1]
            y_true = [real for i in  AveD]
            y_pred =  AveD
            bias = np.mean(y_pred)-np.mean(y_true)
            var = np.var(y_pred)
            nmse = math.sqrt(mean_squared_error(y_true, y_pred))/real
            NMSE.append(nmse);Bias.append(bias);Var.append(var)
        #Degreedist_List.append(Degree_Dist)
        #Density_List.append(Density_SG)

        df_nmse=pd.concat([df_nmse,pd.DataFrame([[np.mean(NMSE)]],columns=['nmse'])])
        df_bias=pd.concat([df_bias,pd.DataFrame([[np.mean(Bias)]],columns=['bias'])])
        df_var=pd.concat([df_var,pd.DataFrame([[np.mean(Var)]],columns=['var'])])
            #df_nmse=pd.DataFrame([[np.mean(NMSE),np.mean(Bias),np.mean(Var)]],columns=['nmse',"bias",'variance'])
            
 
    df_NMSE=pd.concat([df_NMSE,df_nmse], axis=1) 
    df_Bias=pd.concat([df_Var,df_var], axis=1) 
    df_Var=pd.concat([df_Bias,df_bias], axis=1) 

In [240]:
df_NMSE.columns=["ER","SBM",'WS','BA','Graphon']
df_NMSE.index=range(24)
df_NMSE.insert(1,"Method",Name_List)
color_pos=[]
for x in range(0,len(df_NMSE.columns)):
    color_pos.append([df_NMSE.iloc[list(df_NMSE.index),x].abs().sort_values().index[0:3],x])
df_NMSE_=df_NMSE.style.apply(style_specific_cell, axis=None)
df_NMSE_.set_precision(5)
display(df_NMSE_)

TypeError: bad operand type for abs(): 'str'